In [6]:
import numpy as np
import time

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions.normal import Normal

from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.distributions.normal import Normal
from torch.distributions import kl_divergence

In [8]:
def to_scalar(arr):
    if type(arr) == list:
        return [x.item() for x in arr]
    else:
        return arr.item()


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        try:
            nn.init.xavier_uniform_(m.weight.data)
            m.bias.data.fill_(0)
        except AttributeError:
            print("Skipping initialization of ", classname)


class VAE(nn.Module):
    def __init__(self, input_dim, dim, z_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 5, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, z_dim * 2, 3, 1, 0),
            nn.BatchNorm2d(z_dim * 2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(z_dim, dim, 3, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, dim, 5, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, input_dim, 4, 2, 1),
            nn.Tanh()
        )

        self.apply(weights_init)

    def forward(self, x):
        mu, logvar = self.encoder(x).chunk(2, dim=1)

        q_z_x = Normal(mu, logvar.mul(.5).exp())
        p_z = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
        kl_div = kl_divergence(q_z_x, p_z).sum(1).mean()

        x_tilde = self.decoder(q_z_x.rsample())
        return x_tilde, kl_div

In [18]:
BATCH_SIZE = 32
N_EPOCHS = 100
PRINT_INTERVAL = 500
DATASET = 'CIFAR10'  # CIFAR10 | MNIST | FashionMNIST
NUM_WORKERS = 4

INPUT_DIM = 3
DIM = 256
Z_DIM = 128
LR = 1e-3

In [19]:
preproc_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_loader = torch.utils.data.DataLoader(
    eval('datasets.'+DATASET)(
        '../data/{}/'.format(DATASET), train=True, download=True,
        transform=preproc_transform,
    ), batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    eval('datasets.'+DATASET)(
        '../data/{}/'.format(DATASET), train=False,
        transform=preproc_transform
    ), batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

Files already downloaded and verified


In [20]:
model = VAE(INPUT_DIM, DIM, Z_DIM).cuda()
print(model)
opt = torch.optim.Adam(model.parameters(), lr=LR, amsgrad=True)

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(256, 256, kernel_size=(5, 5),

In [21]:
def train():
    train_loss = []
    model.train()
    for batch_idx, (x, _) in enumerate(train_loader):
        start_time = time.time()
        x = x.cuda()

        x_tilde, kl_d = model(x)
        loss_recons = F.mse_loss(x_tilde, x, size_average=False) / x.size(0)
        loss = loss_recons + kl_d

        nll = -Normal(x_tilde, torch.ones_like(x_tilde)).log_prob(x)
        log_px = nll.mean().item() - np.log(128) + kl_d.item()
        log_px /= np.log(2)

        opt.zero_grad()
        loss.backward()
        opt.step()

        train_loss.append([log_px, loss.item()])

        if (batch_idx + 1) % PRINT_INTERVAL == 0:
            print('\tIter [{}/{} ({:.0f}%)]\tLoss: {} Time: {:5.3f} ms/batch'.format(
                batch_idx * len(x), len(train_loader.dataset),
                PRINT_INTERVAL * batch_idx / len(train_loader),
                np.asarray(train_loss)[-PRINT_INTERVAL:].mean(0),
                1000 * (time.time() - start_time)
            ))

In [22]:
def test():
    start_time = time.time()
    val_loss = []
    model.eval()
    with torch.no_grad():
        for batch_idx, (x, _) in enumerate(test_loader):
            x = x.cuda()
            x_tilde, kl_d = model(x)
            loss_recons = F.mse_loss(x_tilde, x, size_average=False) / x.size(0)
            loss = loss_recons + kl_d
            val_loss.append(loss.item())

    print('\nValidation Completed!\tLoss: {:5.4f} Time: {:5.3f} s'.format(
        np.asarray(val_loss).mean(0),
        time.time() - start_time
    ))
    return np.asarray(val_loss).mean(0)

In [23]:
def generate_reconstructions():
    model.eval()
    x, _ = test_loader.__iter__().next()
    x = x[:32].cuda()
    x_tilde, kl_div = model(x)

    x_cat = torch.cat([x, x_tilde], 0)
    images = (x_cat.cpu().data + 1) / 2

    save_image(
        images,
        'samples/vae_reconstructions_{}.png'.format(DATASET),
        nrow=8
    )


def generate_samples():
    model.eval()
    z_e_x = torch.randn(64, Z_DIM, 1, 1).cuda()
    x_tilde = model.decoder(z_e_x)

    images = (x_tilde.cpu().data + 1) / 2

    save_image(
        images,
        'samples/vae_samples_{}.png'.format(DATASET),
        nrow=8
    )

In [26]:
BEST_LOSS = 99999
LAST_SAVED = -1
for epoch in range(1, N_EPOCHS):
    print("Epoch {}:".format(epoch))
    train()
    cur_loss = test()

    if cur_loss <= BEST_LOSS:
        BEST_LOSS = cur_loss
        LAST_SAVED = epoch
        print("Saving model!")
        torch.save(model.state_dict(), 'models/{}_vae.pt'.format(DATASET))
    else:
        print("Not saving model! Last saved: {}".format(LAST_SAVED))

    generate_reconstructions()
    generate_samples()


Epoch 1:


D:\Interface\Anaconda\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


	Iter [15968/50000 (160%)]	Loss: [ 58.5129712  168.72756546] Time: 37.898 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 58.66323599 164.26131366] Time: 37.899 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 58.43319154 156.1838407 ] Time: 36.901 ms/batch

Validation Completed!	Loss: 144.3787 Time: 7.425 s
Saving model!
Epoch 2:
	Iter [15968/50000 (160%)]	Loss: [ 58.2209311  152.88487476] Time: 41.888 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 58.0853374  150.94601231] Time: 37.899 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 57.76300344 145.44314136] Time: 39.893 ms/batch

Validation Completed!	Loss: 135.0530 Time: 7.384 s
Saving model!
Epoch 3:
	Iter [15968/50000 (160%)]	Loss: [ 57.58325534 143.62674252] Time: 39.894 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 57.69323651 143.2014911 ] Time: 38.896 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 57.53444411 139.34386711] Time: 38.895 ms/batch

Validation Completed!	Loss: 129.9874 Time: 7.598 s
Saving model!
Epoch 4:
	Iter [15968/50000 (160%)]	L

Not saving model! Last saved: 24
Epoch 26:
	Iter [15968/50000 (160%)]	Loss: [ 60.46079694 117.46599971] Time: 38.896 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.6792814  118.05843436] Time: 40.891 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.60413248 117.09711987] Time: 37.899 ms/batch

Validation Completed!	Loss: 114.8647 Time: 7.182 s
Saving model!
Epoch 27:
	Iter [15968/50000 (160%)]	Loss: [ 60.47930279 117.2738894 ] Time: 36.901 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.72969827 117.67810599] Time: 38.897 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.6525546  116.84483838] Time: 37.899 ms/batch

Validation Completed!	Loss: 114.6178 Time: 7.061 s
Saving model!
Epoch 28:
	Iter [15968/50000 (160%)]	Loss: [ 60.53734263 117.05899127] Time: 40.890 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.72096124 117.54712762] Time: 39.893 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.63498194 116.70831076] Time: 38.897 ms/batch

Validation Completed!	Loss: 115.2309 Time: 7.311 s
Not sa

	Iter [31968/50000 (320%)]	Loss: [ 60.77705028 113.77979109] Time: 41.888 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.64670212 112.54037614] Time: 40.890 ms/batch

Validation Completed!	Loss: 112.6075 Time: 10.564 s
Saving model!
Epoch 51:
	Iter [15968/50000 (160%)]	Loss: [ 60.52464801 112.83457243] Time: 36.901 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.74801387 113.51090291] Time: 39.892 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.68979105 112.53421844] Time: 37.899 ms/batch

Validation Completed!	Loss: 112.8351 Time: 6.944 s
Not saving model! Last saved: 50
Epoch 52:
	Iter [15968/50000 (160%)]	Loss: [ 60.57089758 112.61052167] Time: 39.895 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.75150619 113.45243686] Time: 41.888 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.66433542 112.33985034] Time: 37.898 ms/batch

Validation Completed!	Loss: 113.0245 Time: 9.071 s
Not saving model! Last saved: 50
Epoch 53:
	Iter [15968/50000 (160%)]	Loss: [ 60.47914889 112.51255963] Time: 37

	Iter [31968/50000 (320%)]	Loss: [ 60.67896583 111.47698386] Time: 37.899 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.59784462 110.38702388] Time: 39.892 ms/batch

Validation Completed!	Loss: 112.1091 Time: 7.025 s
Not saving model! Last saved: 73
Epoch 75:
	Iter [15968/50000 (160%)]	Loss: [ 60.47928478 110.68629259] Time: 38.898 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.66791743 111.32434631] Time: 38.896 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.53323382 110.13509833] Time: 39.893 ms/batch

Validation Completed!	Loss: 112.2387 Time: 7.477 s
Not saving model! Last saved: 73
Epoch 76:
	Iter [15968/50000 (160%)]	Loss: [ 60.39056414 110.57907304] Time: 38.896 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.61330927 111.36669545] Time: 36.901 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.56629232 110.12234514] Time: 38.897 ms/batch

Validation Completed!	Loss: 111.8316 Time: 8.203 s
Saving model!
Epoch 77:
	Iter [15968/50000 (160%)]	Loss: [ 60.444803   110.41875302] Time: 38.

	Iter [31968/50000 (320%)]	Loss: [ 60.57508396 109.87463684] Time: 38.896 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.47492489 108.81749188] Time: 38.896 ms/batch

Validation Completed!	Loss: 112.2460 Time: 7.047 s
Not saving model! Last saved: 87
Epoch 99:
	Iter [15968/50000 (160%)]	Loss: [ 60.33435598 109.31699011] Time: 38.897 ms/batch
	Iter [31968/50000 (320%)]	Loss: [ 60.50001438 109.73557512] Time: 36.900 ms/batch
	Iter [47968/50000 (480%)]	Loss: [ 60.45663988 108.83135912] Time: 48.893 ms/batch

Validation Completed!	Loss: 113.0274 Time: 7.861 s
Not saving model! Last saved: 87
